In [9]:

import pandas as pd
import numpy as np
import metapack as mp
from pathlib import Path
from statsmodels.formula.api import ols
import seaborn as sns
import matplotlib.pyplot as plt
from sdipylib.plot import  source_attribution

# /Users/eric/opt/anaconda3/envs/data/lib/python3.7/site-packages/pandas/plotting/_tools.py:307: MatplotlibDeprecationWarning: 
# The rowNum attribute was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use ax.get_subplotspec().rowspan.start instead.
#   layout[ax.rowNum, ax.colNum] = ax.get_visible()
import warnings
warnings.simplefilter("ignore")

large = 22; med = 16; small = 12
params = {'axes.titlesize': large,
          'legend.fontsize': med,
          'figure.figsize': (16, 10),
          'axes.labelsize': med,
          'axes.titlesize': med,
          'xtick.labelsize': med,
          'ytick.labelsize': med,
          'figure.titlesize': large}
plt.rcParams.update(params)
plt.style.use('seaborn-whitegrid')
sns.set_style("white")
%matplotlib inline

%run weights.py
%run lib.py 

source = "Survey of Consumer Finances, 2016 and 2019 pooled"



In [10]:
pkg = multi_open('federalreserve.gov-consumer_finances-2016e2019-inherit', print_ref=True)
#pkg = mp.jupyter.open_source_package()
pkg

Opening:  index:federalreserve.gov-consumer_finances-2016e2019-inherit


# Survey of Consumer Finances, Inheritance Extract
`federalreserve.gov-consumer_finances-2016e2019-inherit-1.2.5` Last Update: 2020-12-09T02:01:15

_An Extract from the survey of consumer finances for studying networth and ineritance, from multiple years_


This is an extract of the 2016 and 2019 Survey of Consumer Finances, with extracted variables
and recoded variables oriented toward studying inheritance.


 
## Images

[![doc_img](docs/networth_race_inc.png "networth_race_inc")](docs/networth_race_inc.png)

Net Worth by Income Percentile and Race## Notes

* Version 1.2.2 Added the Inflatable flag
* Version 1.2.3 Inflated dollar ammounts to current dollars
## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [inherit_scf_16_19](data/inherit_scf_16_19.csv)**. Extract of the 2016 and 2019 SCF for inheritance and networth

## References
<ul><li> <strong>scf</string>, <em>metapack+http://library.metatab.org/federalreserve.gov-consumer_finances-1.1.1.csv</em>. Survey of consumer finances data packages</li><li> <strong><a href="https://fred.stlouisfed.org/graph/fredgraph.csv?&id=CPIAUCSL">cpi</a></strong>. Consumer Price Index</li><ul>

In [11]:
inherit_scf_16_19 = pkg.resource('inherit_scf_16_19').dataframe()

dfbw,  dfs90 =   make_descriptive_df(inherit_scf_16_19)    

In [14]:
dfs = dfbw.sample(100_000, replace=True, weights=dfbw.wt0)

In [15]:
a = dfs.groupby('race').networth.median().to_frame('median')
c = dfs.groupby('race').networth.mean().to_frame('mean')
t = a.join(c)
display(t)
(t.loc['white']/t.loc['black']).to_frame('ratio').round(2)\
    .join((t.loc['white']-t.loc['black']).to_frame('difference').round(-3).astype(int))


,median,mean
race,,
black,19350.624226,1.757009e+05
white,193162.740059,1.031779e+06


,ratio,difference
median,9.98,174000
mean,5.87,856000


In [22]:
dfs.groupby(['implicate_id', 'race']).networth.mean().unstack().describe()

race,black,white
count,5.000000,5.000000e+00
mean,175858.014531,1.032337e+06
std,48198.158643,8.356299e+04
min,145225.297316,9.556828e+05
25%,146752.227471,9.703432e+05
50%,156385.499631,1.015560e+06
75%,170796.040607,1.055957e+06
max,260131.007627,1.164142e+06


In [26]:
t = dfs.groupby(['implicate_id', 'race']).networth.median().unstack()
t

race,black,white
implicate_id,,
1,18775.334498,194320.405782
2,17095.889439,192727.086112
3,18663.619816,193028.524968
4,21100.719963,194191.217701
5,21984.141133,192598.173607


In [27]:
t.white / t.black

implicate_id
1    10.349771
2    11.273300
3    10.342502
4     9.203061
5     8.760778
dtype: float64